In [0]:
%sql
MERGE INTO gold.transaction AS g
USING (
    WITH btc_price AS (
        SELECT 
            datetime, 
            amount,
            LAG(datetime) OVER (ORDER BY datetime) AS prev_timestamp
        FROM silver.bitcoin_price
    )
    SELECT 
        t.transaction_id,
        t.customer_id,
        t.transaction_type,
        t.btc_amount,
        t.transaction_date,
        ROUND(
            CASE 
                WHEN t.transaction_type = 'compra' THEN (t.btc_amount * p.amount) * -1  
                ELSE (t.btc_amount * p.amount)  
            END, 2
        ) AS transaction_value_in_usd
    FROM silver.transactions t
    JOIN btc_price p
        ON t.transaction_date >= COALESCE(p.prev_timestamp, p.datetime)
        AND t.transaction_date <= p.datetime
) AS s
ON g.transaction_id = s.transaction_id
WHEN MATCHED AND (
    g.customer_id <> s.customer_id OR
    g.transaction_type <> s.transaction_type OR
    g.btc_amount <> s.btc_amount OR
    g.transaction_date <> s.transaction_date OR
    g.transaction_value_in_usd <> s.transaction_value_in_usd
)
THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;